Exclude the records that SATIVA has highligted, redo the alignment and rebuild the tree.

In [1]:
from reprophylo import *

In [2]:
pj=Project('../1-fetch_clean_align/fish_target_locus.csv', pickle='Fish_post_SATIVA.pkl', git=False)

DEBUG:Cloud:Log file (/root/.picloud/cloud.log) opened


In [3]:
pj.read_embl_genbank(['../1-fetch_clean_align/12S_preSATIVA_fish.gb'])

/home/reprophylo/reprophylo.py:1015: UserWarning: Version control off
  warnings.warn('Version control off')


In [4]:
pj.extract_by_locus()

Set the stage:

In [5]:
recs_to_keep = {'12S_r_RNA': []}
recs_to_drop = {'12S_r_RNA': []}

print "to drop: %s" %len(recs_to_drop['12S_r_RNA'])
print "to keep: %s" %len(recs_to_keep['12S_r_RNA'])

print "TOTAL: %s" %(len(recs_to_drop['12S_r_RNA']) + len(recs_to_keep['12S_r_RNA']))

to drop: 0
to keep: 0
TOTAL: 0


Record all record ids and assign them the status keep.

In [6]:
for r in pj.records_by_locus['12S_r_RNA']:
    recs_to_keep['12S_r_RNA'].append("_".join(r.id.split("_")[:-1]))

In [7]:
print "to drop: %s" %len(recs_to_drop['12S_r_RNA'])
print "to keep: %s" %len(recs_to_keep['12S_r_RNA'])

print "TOTAL: %s" %(len(recs_to_drop['12S_r_RNA']) + len(recs_to_keep['12S_r_RNA']))

to drop: 0
to keep: 354
TOTAL: 354


In [8]:
recs_to_drop['12S_r_RNA'].append('AF154850.1')
recs_to_drop['12S_r_RNA'].append('EU048341.1')
recs_to_drop['12S_r_RNA'].append('JN007557.1')
recs_to_drop['12S_r_RNA'].append('JN007558.1')
recs_to_drop['12S_r_RNA'].append('KM052222.1')
recs_to_drop['12S_r_RNA'].append('KU821707.1')
recs_to_drop['12S_r_RNA'].append('DQ447667.1')

print len(recs_to_drop['12S_r_RNA'])

7


In [9]:
for i in range(len(recs_to_drop['12S_r_RNA'])):
    print recs_to_drop['12S_r_RNA'][i]
    for r in range(len(recs_to_keep['12S_r_RNA'])):
        if recs_to_drop['12S_r_RNA'][i] in recs_to_keep['12S_r_RNA'][r]:
            del(recs_to_keep['12S_r_RNA'][r])
            break
print len(recs_to_keep['12S_r_RNA'])

AF154850.1
EU048341.1
JN007557.1
JN007558.1
KM052222.1
KU821707.1
DQ447667.1
347


How many do we want to drop? Parse SATIVA output and extract records ids putatively mislabeled.

In [10]:
rec_mislabelled = {}
sativa=open('../2-SATIVA/12S.mis', 'r')

for line in sativa:
    if not line.startswith(";"):
        rec_mislabelled[line.split("\t")[0]] = line.split("\t")[4]

In [11]:
print len(rec_mislabelled)

7


Adjust lists accordingly.

In [12]:
for r in rec_mislabelled:
    print r
    if not r in recs_to_drop['12S_r_RNA']:
        print "\tadd to droplist"
        recs_to_drop['12S_r_RNA'].append(r)
    for i in reversed(range(len(recs_to_keep['12S_r_RNA']))):
#        print i
        if r in recs_to_keep['12S_r_RNA'][i]:
            print "\tremove from keeplist"
            del(recs_to_keep['12S_r_RNA'][i])

Y12671.1
	add to droplist
	remove from keeplist
KC292943.1
	add to droplist
	remove from keeplist
EU075178.1
	add to droplist
	remove from keeplist
JQ231114.1
	add to droplist
	remove from keeplist
KJ135626.1
	add to droplist
	remove from keeplist
KJ746953.1
	add to droplist
	remove from keeplist
AJ002633.1
	add to droplist
	remove from keeplist


In [13]:
print "to drop: %s" %len(recs_to_drop['12S_r_RNA'])
print "to keep: %s" %len(recs_to_keep['12S_r_RNA'])

print "TOTAL: %s" %(len(recs_to_drop['12S_r_RNA']) + len(recs_to_keep['12S_r_RNA']))

to drop: 14
to keep: 340
TOTAL: 354


Write out good records in full Genbank format including all metadata. Adjust to valid species and taxids before doing it.

Identify records with Subspecies.

In [14]:
from collections import defaultdict

Seqs = []
taxon_to_taxid = {}
recs_to_adjust = defaultdict(list)
taxon_to_recs = defaultdict(list)

for r in pj.records:
    if r.id in recs_to_keep['12S_r_RNA']:
        Seqs.append(r)
        
        source = [f for f in r.features if f.type == 'source'][0]

        #screen for subspecies and record taxids of valid species
        #if valid species record the taxid
        if (len(source.qualifiers['organism'][0].split(" ")) == 2):
            print source.qualifiers['organism'][0],
            if 'db_xref' in source.qualifiers:
#                print source.qualifiers['db_xref']
                for t in source.qualifiers['db_xref']:
#                    print t
                    if 'taxon' in t:
                        if not source.qualifiers['organism'][0] in taxon_to_taxid:
                            print " .. add to records - %s" %t.split(":")[1]
                            taxon_to_taxid[source.qualifiers['organism'][0]] = t.split(":")[1]
                        else:
                            print " .. already covered"
                        taxon_to_recs[source.qualifiers['organism'][0]].append(r.id)
        #if the species status is 'sp.' but has a valid taxid
        elif (len(source.qualifiers['organism'][0].split(" ")) > 2) and ((source.qualifiers['organism'][0].split(" ")[1] == 'sp.') or ('cf.' in source.qualifiers['organism'][0].split(" "))):
            print source.qualifiers['organism'][0],
            if 'db_xref' in source.qualifiers:
                for t in source.qualifiers['db_xref']:
                    if 'taxon' in t:
                        if not source.qualifiers['organism'][0] in taxon_to_taxid:
                            print " .. add to records"
                            taxon_to_taxid[source.qualifiers['organism'][0]] = t.split(":")[1]
                        else:
                            print " .. already covered" 
                        taxon_to_recs[source.qualifiers['organism'][0]].append(r.id)
        #all other cases we consider dubious, e.g. supspecies
        else:
            print "subspecies: %s" %source.qualifiers['organism'][0]
            recs_to_adjust[r.id] = []

Cyprinus carpio  .. add to records - 7962
Cobitis taenia  .. add to records - 98395
subspecies: Cyprinus carpio 'wananensis'
Pungitius pungitius  .. add to records - 134920
Esox lucius  .. add to records - 8010
Cobitis taenia  .. already covered
Hypophthalmichthys molitrix  .. add to records - 13095
Salmo salar  .. add to records - 8030
Oncorhynchus mykiss  .. add to records - 8022
Hypophthalmichthys molitrix  .. already covered
Phoxinus phoxinus  .. add to records - 58324
Salmo salar  .. already covered
Ambloplites rupestris  .. add to records - 109273
Coregonus lavaretus  .. add to records - 59291
Alburnus alburnus  .. add to records - 54556
Hypophthalmichthys molitrix  .. already covered
Salmo salar  .. already covered
Acipenser sturio  .. add to records - 61674
Anguilla anguilla  .. add to records - 7936
Hypophthalmichthys nobilis  .. add to records - 7965
Alburnoides bipunctatus  .. add to records - 58321
Blicca bjoerkna  .. add to records - 58317
Gasterosteus aculeatus  .. add to

Check if any of the taxa that were considered dubious are not already covered in the list after removing subspecies.

In [15]:
from collections import defaultdict

to_fetch = defaultdict(list)

for r in Seqs:
    if r.id in recs_to_adjust:
        source = [f for f in r.features if f.type == 'source'][0]
        adjust_from = source.qualifiers['organism'][0]
        adjust_to = " ".join(adjust_from.split(" ")[:2])
        print "%s -> %s" %(adjust_from,adjust_to)
        if adjust_to in taxon_to_taxid:
            taxon_to_recs[adjust_to].append(r.id)
            recs_to_adjust[r.id].append(adjust_to)
            recs_to_adjust[r.id].append(taxon_to_taxid[adjust_to])
        else:
            to_fetch[adjust_to].append(r.id)
            recs_to_adjust[r.id].append(adjust_to)

Cyprinus carpio 'wananensis' -> Cyprinus carpio
Cyprinus carpio haematopterus -> Cyprinus carpio
Gasterosteus aculeatus williamsoni -> Gasterosteus aculeatus
Carassius auratus auratus -> Carassius auratus
Carassius auratus ssp. 'Pingxiang' -> Carassius auratus
Carassius auratus auratus -> Carassius auratus
Salmo trutta fario -> Salmo trutta
Micropterus salmoides salmoides -> Micropterus salmoides
Salmo trutta fario -> Salmo trutta
Carassius auratus auratus -> Carassius auratus
Carassius auratus auratus -> Carassius auratus
Cyprinus carpio 'wuyuanensis' -> Cyprinus carpio
Carassius auratus ssp. 'Kinbuna' -> Carassius auratus
Abramis brama orientalis -> Abramis brama
Phoxinus phoxinus tumensis -> Phoxinus phoxinus
Salmo trutta trutta -> Salmo trutta
Cyprinus carpio 'color' -> Cyprinus carpio
Carassius auratus auratus -> Carassius auratus
Carassius auratus grandoculis -> Carassius auratus
Gasterosteus aculeatus williamsoni -> Gasterosteus aculeatus
Salmo trutta fario -> Salmo trutta


In [16]:
print len(to_fetch)

0


Modify the records (if necessary) - adjust subspecies to species.

In [17]:
for r in Seqs:
    if r.id in recs_to_adjust:
        print "adjusting %s" %r.id
        source = r.features[0]
        print "%s\t->\t%s" %(source.qualifiers['organism'][0],recs_to_adjust[r.id][0])
        source = r.features[0]
        source.qualifiers['organism'] = [recs_to_adjust[r.id][0]]
        for i in range(len(source.qualifiers['db_xref'])):
            if 'taxon' in source.qualifiers['db_xref'][i]:
                source.qualifiers['db_xref'][i] = "taxon:"+recs_to_adjust[r.id][1]
        if not 'note' in source.qualifiers:
            source.qualifiers['note'] = ['modified from original record: adjusted to species']
        else:
            source.qualifiers['note'].append('modified from original record: adjusted to species')
# print r.format('genbank') 
# print r,recs_to_adjust[r]

adjusting KF856964.1
Cyprinus carpio 'wananensis'	->	Cyprinus carpio
adjusting JX188254.1
Cyprinus carpio haematopterus	->	Cyprinus carpio
adjusting KM273828.1
Gasterosteus aculeatus williamsoni	->	Gasterosteus aculeatus
adjusting AP011236.1
Carassius auratus auratus	->	Carassius auratus
adjusting HQ875340.1
Carassius auratus ssp. 'Pingxiang'	->	Carassius auratus
adjusting GU086395.1
Carassius auratus auratus	->	Carassius auratus
adjusting KP171723.1
Salmo trutta fario	->	Salmo trutta
adjusting HQ391896.1
Micropterus salmoides salmoides	->	Micropterus salmoides
adjusting GU233801.1
Salmo trutta fario	->	Salmo trutta
adjusting AB111951.1
Carassius auratus auratus	->	Carassius auratus
adjusting AB379915.1
Carassius auratus auratus	->	Carassius auratus
adjusting JN105357.1
Cyprinus carpio 'wuyuanensis'	->	Cyprinus carpio
adjusting LC069402.1
Carassius auratus ssp. 'Kinbuna'	->	Carassius auratus
adjusting KC894466.1
Abramis brama orientalis	->	Abramis brama
adjusting KC992395.1
Phoxinus ph

Write only good records with all metadata in Genbank format.

In [18]:
from Bio import SeqIO
       
out = open('../12S_Fish_SATIVA_cleaned_May_2017.gb','w')
SeqIO.write(Seqs, out, "genbank")
out.close()

Exclude the records that were highlighted by SATIVA. Then realign and rebuild phylogeny.

In [19]:
pj.exclude(start_from_max=False, **recs_to_drop)

In [20]:
species=[]
for r in pj.records:
    if r.id in recs_to_keep['12S_r_RNA']:
        source = [f for f in r.features if f.type == 'source'][0]
        species.append(" ".join(source.qualifiers['organism'][0].split()[0:2]))
    
#print set(species)
total=len(species)
species=list(set(species))

print "Total number of records: %i" %total
print "Number of species (see below for full list): %i" %len(species)
print sorted(species)

Total number of records: 340
Number of species (see below for full list): 65
['Abramis brama', 'Acipenser sturio', 'Alburnoides bipunctatus', 'Alburnus alburnus', 'Alosa alosa', 'Alosa fallax', 'Ambloplites rupestris', 'Ameiurus melas', 'Ameiurus nebulosus', 'Anguilla anguilla', 'Barbatula barbatula', 'Barbus barbus', 'Blicca bjoerkna', 'Carassius auratus', 'Carassius carassius', 'Cobitis taenia', 'Coregonus albula', 'Coregonus autumnalis', 'Coregonus lavaretus', 'Coregonus oxyrinchus', 'Cottus gobio', 'Ctenopharyngodon idella', 'Cyprinus carpio', 'Esox lucius', 'Gasterosteus aculeatus', 'Gobio gobio', 'Gymnocephalus cernua', 'Hypophthalmichthys molitrix', 'Hypophthalmichthys nobilis', 'Lampetra fluviatilis', 'Lepomis gibbosus', 'Leucaspius delineatus', 'Leuciscus idus', 'Leuciscus leuciscus', 'Lota lota', 'Micropterus salmoides', 'Misgurnus bipartitus', 'Misgurnus fossilis', 'Neogobius fluviatilis', 'Neogobius melanostomus', 'Oncorhynchus gorbuscha', 'Oncorhynchus mykiss', 'Osmerus ep

In [21]:
fish_list = open("12S_Fish_SATIVA_species_list.txt","w") 
for species_names in species:
  fish_list.write("%s\n" %  species_names)
fish_list.close()

!sort 12S_Fish_SATIVA_species_list.txt > ../12S_Fish_SATIVA_species_list_sort.txt

Pickle the project.

In [22]:
pickle_pj(pj, 'Fish_post_SATIVA.pkl')

'Fish_post_SATIVA.pkl'

#Alignment

Now, need to produce a sequence alignment file based on an alignment of all 12S sequences. This will be done with the AlnConf class.

In [23]:
pj = unpickle_pj('Fish_post_SATIVA.pkl', git=False)

muscle default configuration:

In [24]:
mafft_linsi = AlnConf(pj,                              # The Project
                      method_name='mafftLinsi',        # Any unique method name,
                      CDSAlign=False,                  # Use this method to align
                      #codontable=5,                   # The genetic code that
                      program_name='mafft',            # mafft or muscle.
                      cmd='mafft',                     # The command on your machine
                      loci=['12S_r_RNA'],                    # A list of loci names to align.vel.
                      cline_args={'localpair': True,   # Program specific keywords and arguments.
                                  'maxiterate': 1000}  # cine_args=={} by default, which will
                     )

mafft --localpair --maxiterate 1000 942771494352256.01_12S_r_RNA.fasta


In [25]:
pj.align([mafft_linsi])

In [26]:
print pj.used_methods['mafftLinsi']

AlnConf named mafftLinsi with ID 942771494352256.01
Loci: 12S_r_RNA 
Created on: Tue May  9 17:50:56 2017
Commands:
12S_r_RNA: mafft --localpair --maxiterate 1000 942771494352256.01_12S_r_RNA.fasta

Environment:
Platform: Linux-4.4.0-75-generic-x86_64-with-Ubuntu-14.04-trusty
 Processor: x86_64
 Python build: defaultMar 22 2014 22:59:56
 Python compiler: GCC 4.8.2
 Python implementation: CPython
 Python version: 2.7.6
 ete2 version: 2.2.1072
 biopython version: 1.68
 dendropy version: 3.12.2
 cloud version: 2.8.5
 reprophylo version 1.3
 User: biopc3
 Program and version: MAFFT v7.123b
 Program reference:Katoh
 Standley 2013 (Molecular Biology and Evolution 30:772-780) MAFFT multiple sequence alignment software version 7: improvements in performance and usability.
execution time:
314.60315299


Core Methods section sentence:
The dataset(s) 12S_r_RNA were aligned using the program MAFFT v7.123b [1].

Reference:
Katoh, Standley 2013 (Molecular Biology and Evolution 30:772-780) MAFFT mult

In [27]:
pj.alignments

{'12S_r_RNA@mafftLinsi': <<class 'Bio.Align.MultipleSeqAlignment'> instance (340 records of length 1352, IUPACProtein()) at 7f6c00e2fa10>}

In [28]:
pj.write_alns(id=['record_id','source_organism'], format='fasta')

['12S_r_RNA@mafftLinsi_aln.fasta']

Pickle the project.

In [29]:
pickle_pj(pj, 'Fish_post_SATIVA.pkl')

'Fish_post_SATIVA.pkl'

#Alignment trimming

In [30]:
pj = unpickle_pj('Fish_post_SATIVA.pkl', git=False)

Trim the alignment.

In [31]:
gappyout = TrimalConf(pj,                           # The Project
                      method_name='gappyout',       # Any unique string ('gappyout' is default)
                      program_name='trimal',        # No alternatives in this ReproPhylo version
                      cmd='default',                # default = trimal. Change here or in pj.defaults['trimal']
                      alns=['12S_r_RNA@mafftLinsi'],  # 'all' by default
                      trimal_commands={'gappyout': True} # By default, the gappyout algorithm is used.
                     )

trimal -in 469381494352571.8_12S_r_RNA@mafftLinsi.fasta -gappyout


In [32]:
pj.trim([gappyout])

In [33]:
print pj.used_methods['gappyout']

TrimalConf named gappyout with ID 469381494352571.8
Alignments: 12S_r_RNA@mafftLinsi 
Created on: Tue May  9 17:56:11 2017
Commands:
12S_r_RNA@mafftLinsi@gappyout: trimal -in 469381494352571.8_12S_r_RNA@mafftLinsi.fasta -gappyout

Environment:Platform: Linux-4.4.0-75-generic-x86_64-with-Ubuntu-14.04-trusty
 Processor: x86_64
 Python build: defaultMar 22 2014 22:59:56
 Python compiler: GCC 4.8.2
 Python implementation: CPython
 Python version: 2.7.6
 ete2 version: 2.2.1072
 biopython version: 1.68
 dendropy version: 3.12.2
 cloud version: 2.8.5
 reprophylo version 1.3
 User: biopc3
 Program and version: trimAl 1.2rev59
 Program reference: Salvador Capella-Gutierrez; Jose M. Silla-Martinez; Toni Gabaldon. trimAl: a tool for automated alignment trimming in large-scale phylogenetic analyses. Bioinformatics 2009 25: 1972-1973.
execution time:
2.14150810242

Core Methods section sentence:
The alignment(s) 12S_r_RNA@mafftLinsi were trimmed using the program trimAl 1.2rev59 [1].

Reference:
Sa

In [34]:
pj.trimmed_alignments

{'12S_r_RNA@mafftLinsi@gappyout': <<class 'Bio.Align.MultipleSeqAlignment'> instance (340 records of length 689, IUPACAmbiguousDNA()) at 7f6c0028d6d0>}

In [35]:
pj.write_trimmed_alns(id=['record_id','source_organism'], format='fasta')

['12S_r_RNA@mafftLinsi@gappyout_trimmed_aln.fasta']

Trimmed alignments can be viewed in the same way as alignments, but using this command:

In [36]:
pj.show_aln('12S_r_RNA@mafftLinsi@gappyout',id=['source_organism'])

Pickle the project.

In [37]:
pickle_pj(pj, 'Fish_post_SATIVA.pkl')

'Fish_post_SATIVA.pkl'

## Build tree

In [38]:
pj = unpickle_pj('Fish_post_SATIVA.pkl', git=False)

Infer Maximum Likelihood tree based on the clipped alignment. We are going to use RAxML. 

Configure RAxML for trimmed mafft linsi alignment.

In [39]:
raxml = RaxmlConf(pj, 
                  method_name='raxml-mafftLinsi-SATIVA-trimmed',
                  program_name='raxmlHPC-PTHREADS-SSE3',
                  keepfiles=True,
                  cmd='raxmlHPC-PTHREADS-SSE3',
                  preset='fa',
                  alns=['12S_r_RNA@mafftLinsi@gappyout'],
                  model='GAMMA',
                  threads=5,
                  cline_args={'-#': 100})

raxmlHPC-PTHREADS-SSE3 -f a -m GTRGAMMA -n 302421494352575.49_12S_r_RNA@mafftLinsi@gappyout0 -p 787 -s 302421494352575.49_12S_r_RNA@mafftLinsi@gappyout.fasta -T 5 -x 886 -N 100


Infer tree.

In [40]:
pj.tree([raxml])

In [41]:
print pj.used_methods['raxml-mafftLinsi-SATIVA-trimmed']

RaxmlConf named raxml-mafftLinsi-SATIVA-trimmed with ID 302421494352575.49
Alignments: 12S_r_RNA@mafftLinsi@gappyout 
Created on: Tue May  9 17:56:15 2017
Commands:
12S_r_RNA@mafftLinsi@gappyout:
raxmlHPC-PTHREADS-SSE3 -f a -m GTRGAMMA -n 302421494352575.49_12S_r_RNA@mafftLinsi@gappyout0 -p 787 -s 302421494352575.49_12S_r_RNA@mafftLinsi@gappyout.fasta -T 5 -x 886 -N 100

Environment:
Platform: Linux-4.4.0-75-generic-x86_64-with-Ubuntu-14.04-trusty
 Processor: x86_64
 Python build: defaultMar 22 2014 22:59:56
 Python compiler: GCC 4.8.2
 Python implementation: CPython
 Python version: 2.7.6
 ete2 version: 2.2.1072
 biopython version: 1.68
 dendropy version: 3.12.2
 cloud version: 2.8.5
 reprophylo version 1.3
 User: biopc3
 Program and version: raxmlHPC-PTHREADS-SSE3RAxML version 8.0.12
 Program reference: A. Stamatakis: RAxML Version 8: A tool for Phylogenetic Analysis and Post-Analysis of Large Phylogenies. In Bioinformatics
 2014.
execution time:
1671.50975013

Core Methods section s

In [42]:
pickle_pj(pj, 'Fish_post_SATIVA.pkl')

'Fish_post_SATIVA.pkl'

# Visualise Tree

Now let's visualize the tree.Prepare tree annotation:

- define outgroups
- specify support coloration

In [43]:
pj.add_qualifier_from_source('organism')

In [44]:
supports = {'black': [100,95],
            'dimgray': [95,75],
            'silver': [75,50]}

In [45]:
for species in ['Lampetra']:
    pj.if_this_then_that(species, 'organism', 'outgroup', 'outgroup',
                         mode='part')

In [46]:
pj.clear_tree_annotations()

pj.annotate('.', 'outgroup','outgroup',
            ['source_organism', 'record_id'], 
            node_support_dict=supports, multifurc=10,
            scale=10, ##This will determine the width of the tree (default 1000)
            html='mafftLinsi-fish-trimmed-raxml-post_SATIVA.html')

Create a static copy of the tree (the last *.png with the expected alignment id) for later.

In [47]:
%%bash

last_png=$(ls -hlrt *12S_r_RNA@mafftLinsi@gappyout.png | tail -n 1 | perl -ne 'chomp; @a=split(" "); print "$a[-1]"')
cp $last_png 12S_mafftLinsi-fish-trimmed-raxml-post_SATIVA.png

Pickle the project.

In [48]:
pickle_pj(pj, 'Fish_post_SATIVA.pkl')

'Fish_post_SATIVA.pkl'

**=======================================================================================================**